# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np

from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, accuracy_score, mean_absolute_error, mean_squared_error, roc_curve, auc, confusion_matrix, classification_report

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

# Reading the Dataset

In [ ]:
!ls ../input/indian-liver-patient-records/indian_liver_patient.csv

In [ ]:
data=pd.read_csv("../input/indian-liver-patient-records/indian_liver_patient.csv")
data

# Data Preprocessing

In [ ]:
data.isnull().sum()

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
data["Albumin_and_Globulin_Ratio"]=data["Albumin_and_Globulin_Ratio"].bfill()

In [ ]:
data.isnull().sum()

In [ ]:
data["Dataset"].value_counts()

In [ ]:
x=data.iloc[:,:-1]
y=data.iloc[:,-1]
y

In [ ]:
#Balancing the class
oversample = RandomOverSampler()
x, y= oversample.fit_resample(x, y)
type(x)

In [ ]:
new_data=pd.DataFrame(data=x, columns=x.columns)
new_data["Dataset"]=y
new_data

In [ ]:
#Now class is balanced
new_data["Dataset"].value_counts()

In [ ]:
new_data.shape

In [ ]:
#Converting object to int for processing
la= LabelEncoder()
new_data["Gender"]=la.fit_transform(new_data["Gender"])

# Data Visualization

In [ ]:
corr=new_data.corr()
plt.subplots(figsize=(12,6))
sns.heatmap(corr, annot=True)

In [ ]:
sns.pairplot(new_data)

In [ ]:
corr["Dataset"].sort_values(ascending=False)

# Data Splitting

In [ ]:
x=new_data.iloc[:,:-1]
sc= StandardScaler()
sc.fit_transform(x)
y=new_data["Dataset"]
x

In [ ]:
xtrain, xtest, ytrain, ytest= train_test_split(x, y, test_size=0.25, random_state=42)

# Applying Model

In [ ]:
cla= KNeighborsClassifier(n_neighbors=5)
cla.fit(xtrain, ytrain)
cla.get_params()

In [ ]:
param={
    'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute'],
    'n_neighbors': [ i for i in range (5,15,2)],
    'weights':['uniform', 'distance'] 
}

In [ ]:
grid = GridSearchCV(estimator=cla, param_grid=param, cv= 5)
grid.fit(xtrain, ytrain)

In [ ]:
# Finding best parameters
grid.best_params_

In [ ]:
cla= KNeighborsClassifier(n_neighbors=13, weights='distance', algorithm='auto')
cla.fit(xtrain, ytrain)

# Checking Accuracy

In [ ]:
ytrain_pre=cla.predict(xtrain)
print("Training accuracy is {} %".format(accuracy_score(ytrain, ytrain_pre)*100))

In [ ]:
ypredict=cla.predict(xtest)
print("Testing accuracy is {} %".format(accuracy_score(ytest, ypredict)*100))

In [ ]:
pd.DataFrame({"Actual":ytest,"Predicted":ypredict})

In [ ]:
a = confusion_matrix(ytest, ypredict)
sns.heatmap(a, annot=True)

In [ ]:
print("KNN Model Classification Report")
print(classification_report(ytest, ypredict))